In [1]:
WaveRNN_path = '/home/erogol/projects/'

In [2]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import sys
sys.path.append(WaveRNN_path)
from TTS.utils.audio import AudioProcessor
from TTS.utils.generic_utils import load_config
import glob 
import IPython.display as ipd

In [3]:
config_path = "/home/erogol/projects/WaveRNN/config_mozilla.json"
data_path = "/home/erogol/Data/LJSpeech-1.1/wavs/"
file_paths = glob.glob(data_path + "/**/*.wav", recursive=True)
CONFIG = load_config(config_path)

In [4]:
audio={
 'audio_processor': 'audio',
 'num_mels': 80,          # In general, you don'tneed to change it 
 'num_freq': 1025,        # In general, you don'tneed to change it 
 'sample_rate': 22050,    # It depends to the sample rate of the dataset.
 'frame_length_ms': 50,   # In general, you don'tneed to change it 
 'frame_shift_ms': 12.5,  # In general, you don'tneed to change it 
 'preemphasis': 0.98,        # In general, 0 gives better voice recovery but makes traning harder. If your model does not train, try 0.97 - 0.99.
 'min_level_db': -100,
 'ref_level_db': 20,      # It is the base DB, higher until you remove the background noise in the spectrogram and then lower until you hear a better speech below.
 'power': 1.5,            # Change this value and listen the synthesized voice. 1.2 - 1.5 are some resonable values.
 'griffin_lim_iters': 60, # It does not give any imporvement for values > 60
 'signal_norm': True,     # This is more about your model. It does not give any change for the synthsis performance.
 'symmetric_norm': False,   # Same as above
 'max_norm': 1,           # Same as above
 'clip_norm': True,       # Same as above
 'mel_fmin': 50.0,        # You can play with this and check mel-spectrogram based voice synthesis below.
 'mel_fmax': 8000.0,        # You can play with this and check mel-spectrogram based voice synthesis below.
 'do_trim_silence': True} # If you dataset has some silience at the beginning or end, this trims it. Check the AP.load_wav() below,if it causes any difference for the loaded audio file.

AP = AudioProcessor(**audio);

 > Setting up Audio Processor...
 | > bits:None
 | > sample_rate:22050
 | > num_mels:80
 | > min_level_db:-100
 | > frame_shift_ms:12.5
 | > frame_length_ms:50
 | > ref_level_db:20
 | > num_freq:1025
 | > power:1.5
 | > preemphasis:0.98
 | > griffin_lim_iters:60
 | > signal_norm:True
 | > symmetric_norm:False
 | > mel_fmin:50.0
 | > mel_fmax:8000.0
 | > max_norm:1.0
 | > clip_norm:True
 | > do_trim_silence:True
 | > n_fft:2048
 | > hop_length:275
 | > win_length:1102


In [5]:
import random
idx = random.randint(0, len(file_paths))
wav = AP.load_wav(file_paths[idx])
ipd.Audio(data=wav, rate=AP.sample_rate) 

In [6]:
quant = AP.mulaw_encode(wav, 9)
assert quant.max() < 2 ** 9
assert quant.min() >= 0
wav_hat = AP.mulaw_decode(quant, 9)
ipd.Audio(data=wav_hat, rate=AP.sample_rate) 

In [7]:
print(wav_hat.min())
print(wav_hat.max())
print(wav_hat.mean())

-0.906771
0.8848518
1.294747e-05


In [8]:
print(quant.min())
print(quant.max())
print(quant.mean())

4.0
506.0
248.91695222508255
